In [3]:
import pandas as pd
import os, glob, re
import PIL.ExifTags as ExifTags
import PIL.Image as Image

# Naming conventions

In [4]:
all_jpgs= glob.glob('C:/Users/student1/Desktop/Datasets/dress3_16_small/input/*.JPG')
print(f"total images: {len(all_jpgs)}")

total images: 139


In [ ]:
def get_exif(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if not exif_data:
        return {}

    exif = {}
    for tag, value in exif_data.items():
        decoded = ExifTags.TAGS.get(tag, tag)
        exif[decoded] = value
    return exif

def get_exif_gps(exif_info: dict):
    gps_info = {}

    for key in exif_info['GPSInfo'].keys():
        decode = ExifTags.GPSTAGS.get(key,key)
        gps_info[decode] = exif_info['GPSInfo'][key]#int.from_bytes(exif_info['GPSInfo'][key], byteorder='big')
    return gps_info


exif_info = get_exif(all_jpgs[0])
if exif_info == {}:
    print("No EXIF data found for image")
    exit()
    
for tag, value in exif_info.items():

    if tag == 'GPSInfo':
        print("GPSInfo")
        gps_info = get_exif_gps(exif_info)
        for tag, value in gps_info.items():
            print(f"{' ' * 5}{tag}: {value}")

    else:
        print(f"{tag}: {value}")
    

No EXIF data found for image


: 

In [20]:

# make a dataframe with 3 columns
camera_names_df= pd.DataFrame(columns=['tower', 'aim', 'name', 'focal_length', 'Width', 'Height', 'Camera_model'])


for i, jpg in enumerate(all_jpgs):
    camera_name= re.split('[-_]', os.path.basename(jpg))[2]
    exif_info = get_exif(jpg)
    
    # Create a dictionary with the new data
    try:
        camera_names_df.loc[len(camera_names_df)] = {
            'tower': camera_name[0],
            'aim': camera_name[1],
            'name': camera_name[2],
            'focal_length': exif_info['FocalLength'],
            'Width': exif_info['ExifImageWidth'],
            'Height': exif_info['ExifImageHeight'],
            'Camera_model': exif_info['LensModel']
        }
    except KeyError:
        print(f"KeyError: {i}, {jpg}")
        
    # if camera_name == 'KF1':
    #     print(i, exif_info['LensModel'])
    
# change i to I
camera_names_df['tower']= camera_names_df['tower'].str.upper()


KeyError: 0, C:/Users/student1/Desktop/Datasets/dress3_16_small/input\marie_PL-AB1_dress3_16.JPG
KeyError: 1, C:/Users/student1/Desktop/Datasets/dress3_16_small/input\marie_PL-AB2_dress3_16.JPG
KeyError: 2, C:/Users/student1/Desktop/Datasets/dress3_16_small/input\marie_PL-AB3_dress3_16.JPG
KeyError: 3, C:/Users/student1/Desktop/Datasets/dress3_16_small/input\marie_PL-AB4_dress3_16.JPG
KeyError: 4, C:/Users/student1/Desktop/Datasets/dress3_16_small/input\marie_PL-AB5_dress3_16.JPG
KeyError: 5, C:/Users/student1/Desktop/Datasets/dress3_16_small/input\marie_PL-AB6_dress3_16.JPG
KeyError: 6, C:/Users/student1/Desktop/Datasets/dress3_16_small/input\marie_PL-AB7_dress3_16.JPG
KeyError: 15, C:/Users/student1/Desktop/Datasets/dress3_16_small/input\marie_PL-BB1_dress3_16.JPG
KeyError: 16, C:/Users/student1/Desktop/Datasets/dress3_16_small/input\marie_PL-BB3_dress3_16.JPG
KeyError: 17, C:/Users/student1/Desktop/Datasets/dress3_16_small/input\marie_PL-BB4_dress3_16.JPG
KeyError: 18, C:/Users/stud

In [10]:
camera_names_df.to_csv('camera_names.csv', index=False)

In [11]:
# group by  width and height then focal length

group_df= pd.pivot_table(camera_names_df, index=['Width', 'Height', 'focal_length', 'tower', 'aim'], aggfunc='count')
display(group_df)

Camera_model  name
Width Height focal_length tower aim                    
4272  2848   27.0         K     F               1     1
5184  3456   50.0         H     B               1     1
6240  4160   35.0         A     B               1     1
                          E     B               1     1
                          H     B               1     1
                          K     B               1     1
             50.0         A     B               6     6
                          B     B               8     8
                          C     B               6     6
                          D     B               8     8
                          E     B               6     6
                          F     B               8     8
                          G     B               6     6
                          H     B               2     2
                          I     B               7     7
                          J     B               4     4
                          K     B               7     7
             85.0         C     F               2     2
                          E     F               2     2
                          G     F               2     2
                          K     F               1     1

In [34]:
camera_names_df[((camera_names_df["tower"]=="A") & (camera_names_df["aim"]=="B"))]

,tower,aim,name,focal_length
0,A,B,1,50.0
1,A,B,2,50.0
2,A,B,3,50.0
3,A,B,4,50.0
4,A,B,5,50.0
5,A,B,6,35.0
6,A,B,7,50.0
7,A,B,M,50.0
